<br>

# MapBiomas

[![Repo](https://img.shields.io/badge/GitHub-repo-blue?logo=github&logoColor=f5f5f5)](https://github.com/mapbiomas-brazil)

O [MapBiomas](https://brasil.mapbiomas.org/) é uma rede colaborativa de co-criadores, composta por ONGs, universidades e empresas de tecnologia, que tem como objetivo principal mapear anualmente a cobertura e o uso da terra no Brasil e monitorar as mudanças no território. Essa iniciativa do Observatório do Clima utiliza tecnologia de alta qualidade e baixo custo para produzir séries anuais de mapas desde 1985 até o presente.

<br>

**Como o MapBiomas funciona?**

O MapBiomas funciona através de um processo automatizado e distribuído, utilizando a plataforma Google Earth Engine. As principais etapas de seu funcionamento incluem:

- Coleta de Imagens de Satélite: Utiliza grandes volumes de imagens de satélite, principalmente da série Landsat, que registram o planeta desde 1985. Essas imagens são capturadas periodicamente (a cada 15-17 dias), fornecendo uma base de dados histórica e atualizada.
- Processamento e Classificação: Divide o território brasileiro em bilhões de "pedacinhos" (pixels de 30x30 metros). Para cada um desses pixels, algoritmos de inteligência artificial são aplicados para classificar o tipo de cobertura e uso da terra (ex: floresta, pastagem, agricultura, água, área urbana, etc.) em cada ano da série histórica.
- Filtro Temporal: Aplica regras de filtro temporal para garantir a consistência e a precisão dos dados ao longo dos anos, minimizando erros e flutuações temporárias.
- Integração de Mapas: Os mapas de cada classe de cobertura e uso são integrados em um único mapa que representa o estado do território brasileiro para cada ano.
- MapBiomas Alerta: Uma vertente do MapBiomas, o MapBiomas Alerta, valida e refina alertas de desmatamento gerados por outros sistemas de monitoramento (como o DETER do INPE), utilizando imagens de alta resolução e inteligência artificial. Isso gera laudos detalhados que auxiliam a fiscalização ambiental.

<br>

---

**Qual a importância do MapBiomas?**

A importância do MapBiomas é multifacetada e crucial para a gestão ambiental e o desenvolvimento sustentável do Brasil:

- Monitoramento Ambiental: Fornece dados confiáveis e atualizados sobre as mudanças na cobertura e uso da terra, permitindo o monitoramento contínuo do desmatamento, da regeneração florestal, da expansão agrícola, da mineração, entre outros.
- Subsídio a Políticas Públicas: As informações geradas são essenciais para a formulação e implementação de políticas públicas relacionadas à conservação, ao combate às mudanças climáticas e à regularização ambiental (como o Cadastro Ambiental Rural - CAR).
- Fiscalização e Combate a Crimes Ambientais: Através do MapBiomas Alerta, o projeto oferece informações detalhadas sobre desmatamentos, que são utilizadas por órgãos fiscalizadores (IBAMA, ICMBio, Secretarias de Meio Ambiente) para planejar ações de fiscalização e combater o desmatamento ilegal.
- Transparência e Acesso à Informação: Os dados do MapBiomas são abertos e acessíveis ao público, permitindo que pesquisadores, gestores, sociedade civil e cidadãos em geral possam acessar e utilizar essas informações para análise, pesquisa e tomada de decisões informadas.
- Pesquisa Científica e Acadêmica: Os dados do MapBiomas são amplamente utilizados em pesquisas científicas e estudos acadêmicos sobre mudanças ambientais, uso do solo e impactos climáticos.
- Inovação Tecnológica: O projeto demonstra o potencial do uso de sensoriamento remoto, inteligência artificial e plataformas de computação em nuvem para o monitoramento ambiental em larga escala.

<br>

Em suma, o MapBiomas é uma ferramenta estratégica que democratiza o acesso a informações ambientais de alta qualidade, contribuindo significativamente para a conservação dos recursos naturais e para o enfrentamento dos desafios das mudanças climáticas no Brasil.


In [ ]:
#!pip3 install geemap --upgrade

In [ ]:
#import pprint
#from datetime import date, datetime, timedelta

# import geemap
import pprint

import ee
import folium
import geemap
import pandas as pd

import open_geodata as geo

#import geopandas as gpd
#import IPython.display as disp
#import matplotlib.pyplot as plt
#import numpy as np

#from scipy.stats import chi2, f, gamma, norm

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
ee.Initialize(project='ee-python-michel')

<br>

-----

## Coleção 9

In [ ]:
image = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1')
image.getInfo()

In [ ]:
for k in image.getInfo()['bands']:
    lyr = k['id']
    print(lyr)

In [ ]:
image = image.select('classification_2023')
image.getInfo()

In [ ]:
lyr

<br>

-----

## Legenda

In [ ]:
db = geo.data.DB(db='map_biomas')
db.json

In [ ]:
filepath = db.get_data(name='tab.codigos-da-legenda-colecao-9')
filepath

In [ ]:
df = pd.read_csv(filepath_or_buffer=filepath, sep='\t')
df = df.sort_values(by='Class_ID')
df.head()

In [ ]:
legend_dict = dict(zip(df['Descricao'], df['Color']))

[Descritores de camadas estilizados](https://developers.google.com/earth-engine/guides/image_visualization?hl=pt-br#styled-layer-descriptors)

https://geemap.org/notebooks/14_legends/

In [ ]:
list_str = []
list_str.append('<RasterSymbolizer>')
list_str.append('<ColorMap  type="intervals" extended="false" >')
for index, row in df.iterrows():
    id_color = row['Class_ID']
    color = row['Color']
    descricao = row['Descricao']

    list_str.append(
        f'<ColorMapEntry color="{color}" quantity="{id_color}" label="{descricao}"/>'
    )


list_str.append('</ColorMap>')
list_str.append('</RasterSymbolizer>')
sld_intervals = '\n'.join(list_str)
pprint.pprint(sld_intervals)

In [ ]:
# Apply the SLD style to the image.
cover_sld = image.sldStyle(sld_intervals)

# Define a map centered on the United States.
m = geemap.Map(center=[-10, -48], zoom=5)

# Add the image layer to the map and display it.
m.add_layer(cover_sld, None, 'MapBiomas')

m.add_legend(
    title='MapBiomas Classification',
    legend_dict=legend_dict,
    layer_name='MapBiomas',
)

display(m)

: 